# Trace your application with Azure AI Foundry
Inspiration: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-local-sdk?tabs=python

## Documentation

Tracing in Azure AI Foundry - https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/trace

Inference 
- https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/concepts/endpoints?tabs=python
- https://learn.microsoft.com/en-us/python/api/overview/azure/ai-inference-readme?view=azure-python-preview
- https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/how-to/configure-entra-id?tabs=python&pivots=ai-foundry-portal#configure-microsoft-entra-id-for-inference

OpenTelemetry 
- https://learn.microsoft.com/en-us/azure/azure-monitor/app/opentelemetry-enable?tabs=
- https://learn.microsoft.com/en-us/azure/azure-monitor/app/opentelemetry-help-support-feedback?tabs=aspnetcore#why-is-microsoft-azure-monitor-investing-in-opentelemetry


## Setup

### Common packages

In [103]:
import os
import dotenv
from pathlib import Path

### Global settings

In [104]:
# Global variables
PRIVATE = False
DATA_DIR = Path("data")
TMP_DIR = Path("tmp")

### Load environment variables

In [105]:
# Import override environment variables from .env file
# or from private.env file if PRIVATE is True
dotenv.load_dotenv('.env' if not PRIVATE else 'private.env', override=True)

True

### Azure credentials

In [106]:
# https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

### Get Azure AI Foundry project client

In [107]:
from azure.ai.projects import AIProjectClient

# Create an Azure AI Client from a connection string. Available on Azure AI project Overview page.
# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.aiprojectclient?view=azure-python-preview
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ.get("AZURE_AI_PROJECT_CONNECTION_STRING"),
)

## Configure Tracing to Application Insights

### Activate content recording 

In [108]:
os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'


### Check APplication insights is enabled for this Azure AI Foundry project

In [109]:
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("WARNING: Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")


### Configure Azure Monitor using the AI projects connection string 

In [110]:
from azure.monitor.opentelemetry import configure_azure_monitor

# https://learn.microsoft.com/en-us/python/api/azure-monitor-opentelemetry/azure.monitor.opentelemetry?view=azure-python    
configure_azure_monitor(connection_string=application_insights_connection_string)

### Use the generic Azure AI inference API for models and tracing

#### Essential dependency
- User needs EXACTLY "Cognitive Services User" on the Azure AI Services resource. 
- "Cognitive Services Contributor" will not work, although it gives more rights!

In [111]:
from azure.ai.inference import ChatCompletionsClient

# https://learn.microsoft.com/en-us/python/api/azure-ai-inference/azure.ai.inference.chatcompletionsclient?view=azure-python-preview
client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_AI_INFERENCE_ENDPOINT"],
    credential=credential,
    credential_scopes=["https://cognitiveservices.azure.com/.default"],
)


## Run arbitrary AI application to check out tracing

In [112]:
# https://learn.microsoft.com/en-us/python/api/azure-ai-inference/azure.ai.inference.models.systemmessage?view=azure-python-preview
# https://learn.microsoft.com/en-us/python/api/azure-ai-inference/azure.ai.inference.models.usermessage?view=azure-python-preview
from azure.ai.inference.models import SystemMessage, UserMessage

# https://learn.microsoft.com/en-us/python/api/azure-ai-inference/azure.ai.inference.chatcompletionsclient?view=azure-python-preview#methods
response = client.complete(
    model=os.environ["AZURE_AI_DEPLOYMENT"],
    messages=[
        # SystemMessage(content="You are a helpful assistant."),
        SystemMessage(content="You're a chatbot that only responds in whimsical rhyme!"),
        UserMessage(content="How many languages are in the world?"),
    ]
)

print(response.choices[0].message.content)

Oh, dear traveler, let me unfurl,  
The mystery of languages in our world!  
There are many—thousands, they say,  
Close to 7,000 at last survey.  

From whispers in forests to city sounds bright,  
Each one a star in linguistic night.  
Though some fade away and few take the stage,  
Their stories are treasures of history’s age!  
